In [ ]:
from imports import *
from functions_uc import *

def timellm_predict(full_data,scaler,prediction_length,context_length,prompt_prefix):
    full_data_test = full_data.groupby("id").tail(prediction_length)
    full_data_train = full_data.drop(full_data_test.index)
    full_data_train_ = full_data_train.rename(columns={'date': 'ds', 'id': 'unique_id','value':'y'})
    full_data_test_ = full_data_test.rename(columns={'date': 'ds', 'id': 'unique_id','value':'y'})
    full_data= full_data.rename(columns={'date': 'ds', 'id': 'unique_id','value':'y'})
    full_data_train_ = full_data_train_.drop(columns=['type'])
    full_data_test_ = full_data_test_.drop(columns=['type'])
    random.seed(42)
    n_observed = full_data_test_['unique_id'].nunique()
    random_encounter_ids = random.sample(list(full_data_train_['unique_id']), n_observed) # change for no fine-tuning
    # Filtering out the selected encounter IDs
    full_data_train_limit = full_data_train_[full_data_train_['unique_id'].isin(random_encounter_ids)]
    full_data_test_limit = full_data_test_[full_data_test_['unique_id'].isin(random_encounter_ids)]
    prompt_prefix = prompt_prefix
    timellm = TimeLLM(
        h=prediction_length,
        input_size=context_length,
        llm='meta-llama/Llama-2-7b-hf',
        prompt_prefix=prompt_prefix,
        batch_size=24,
        d_llm=4096,
        valid_batch_size=24,
        windows_batch_size=24,
        max_steps=100,
        learning_rate=1e-4
    )

    nf = NeuralForecast(
        models=[timellm],
        freq='D'
        )
    nf.fit(df=full_data_train_limit)
    forecasts = nf.predict(full_data_train_)
    
    def rescale_to_original(scaled_df, scaler, columns_to_rescale):
        scaled_df[columns_to_rescale] = scaler.inverse_transform(scaled_df[columns_to_rescale])
        return scaled_df
    
    columns_to_rescale_actual = ["y"]
    columns_to_rescale_predict = ["TimeLLM"]
    actuals = rescale_to_original(full_data_test_, scaler,columns_to_rescale_actual)
    predictions = rescale_to_original(forecasts, scaler,columns_to_rescale_predict)
    return predictions,actuals,nf